In [ ]:
import csv
import os
import json
import argparse
import random
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Loading

In [ ]:
scamdigger_dataset_path = "/content/drive/MyDrive/NUS MSBA/CS5344/script/scamdigger/scam_profiles_output.csv" #2021 onwards
datingnmore_dataset1_path = "/content/drive/MyDrive/NUS MSBA/CS5344/script/datingnmore/result_1_501.csv"
datingnmore_dataset2_path = "/content/drive/MyDrive/NUS MSBA/CS5344/script/datingnmore/result_500_1000.csv" #2018 onwards

scam_df = pd.read_csv(scamdigger_dataset_path)
dating_df1 = pd.read_csv(datingnmore_dataset1_path)
datingn_df2 = pd.read_csv(datingnmore_dataset1_path)

scam_df.shape, dating_df1.shape, datingn_df2.shape
# print(scam_df.dtypes)

((2622, 17), (15030, 17), (15030, 17))

In [ ]:
datingnmore_df = pd.concat([dating_df1, datingn_df2], ignore_index=True)
datingnmore_df.shape

(30060, 17)

In [ ]:
scam_df.head()

,year_reported,month_reported,Username,Email,Name,Age,Purpose,Looking,Location,Status,Children,Orientation,Ethnicity,Religion,Occupation,Description,IP
0,2024.0,1.0,leesung212,"davilageorge900@gmail.com, ubaubaifeanyi27@gma...","Davila George, Lee Sung",47 y.o.,"Friendship, Romance, Serious Relationship, Mar...","Male, Female, Both","Texas, United States",Widowed,I have 1-2 children living with me,Bisexual,Asian,Other,Doctor,NaN,address 197.210.78.85 – Nigeria
1,2024.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024.0,1.0,ArmandPaillarde4,planamonique5@gmail.com,Armand Paillarde,50 y.o.,Serious Relationship,Female,"bruxelle, France",Single,I don’t have children,Straight,White,Christian,NaN,look into the future with hope and would like ...,address 41.207.22.144 – Ivory Coast
3,2024.0,1.0,JuanLopez1956,donaldson555666@gmail.com,Juan Lopez,52 y.o.,Marriage,Female,"Michigan, United States",Single,I have 1-2 children living with me,Straight,Other,Christian,Doctor,Medical Surgeon. Humble and kind hearted,address 41.66.202.159 – Ghana
4,2024.0,1.0,Ethan123,ethanmarc253@gmail.com,Ethan Marc,29 y.o.,"Romance, Serious Relationship, Marriage",Male,"Saint-Louis, France",Single,I don’t have children,Bisexual,White,Christian,Architecte,NaN,address 185.104.185.114 – proxy


In [ ]:
datingnmore_df.head()

,name,gender,age,location,status,username,ethnicity,occupation,description,match_age,children,orientation,religion,smoking,drinking,intent,target
0,LBreeze,female,63 y.o.,"Spokane, WA, USA",single,LBreeze,white,Livein caregiver,I am an extremely gentle hearted kind loving s...,from 19 to 86,1-2 living elsewhere,Straight,Spiritual,light smoker,occasional drinker,"Friendship, Romance, Serious Relationship, Mar...",Male
1,Rossiii,female,27 y.o.,"Louisville, KY, USA",single,Rossiii,white,-,-,from 19 to 27,no children,Straight,Other,non-smoker,never,"Fun, Friendship, Romance, Serious Relationship...",Male
2,Lendys,male,36 y.o.,"Bogotá, Bogota, Colombia",single,Lendys,hispanic,Ingeniero,Ingeniero en sistemas latino con un gran senti...,from 28 to 35,no children,Straight,Christian,non-smoker,occasional drinker,Romance,Female
3,DanielVega,male,29 y.o.,"Lima, Perú",single,DanielVega,mixed,Ingeniero y pianista,¡Paz y bien!\r\nSoy un hombre que ama al Señor...,from 19 to 30,want children,Straight,Christian,non-smoker,never,"Friendship, Serious Relationship, Marriage",Female
4,Pickles69,female,55 y.o.,"Bloomington, IL, USA",divorced,Pickles69,white,Employed Full time,-,from 55 to 65,1-2 living elsewhere,Straight,Other,non-smoker,social drinker,"Fun, Friendship, Romance, Serious Relationship...",Male


In [ ]:
scam_df.columns = scam_df.columns.str.lower()
scam_df = scam_df.rename(columns = {'looking':'target','purpose':'intent'})


In [ ]:
scam_df['age'] = scam_df['age'].fillna('').astype(str).str.extract(r'(\d+)').astype(float)
datingnmore_df['age'] = datingnmore_df['age'].str.extract(r'(\d+)').astype(float)
scam_df['status'] = scam_df['status'].str.lower()
datingnmore_df = datingnmore_df[datingnmore_df['ethnicity'] != 'error']
scam_df['ethnicity'] = scam_df['ethnicity'].str.lower() #fill in nan value with mod later
scam_df['occupation'] = scam_df['occupation'].str.lower()
datingnmore_df['occupation'] = datingnmore_df['occupation'].str.lower() #fill '-' with mod later
datingnmore_df['location'] = datingnmore_df['location'].str.lower()
scam_df['location'] = scam_df['location'].str.lower()

In [ ]:
scam_df.loc[scam_df['children'] == 'I have 1-2 children living with me','children'] = '1-2 living with me'
scam_df.loc[scam_df['children'] == 'I have 1-2 children living elsewhere','children'] = '1-2 living elsewhere'
scam_df.loc[scam_df['children'] == 'I don’t have children','children'] = 'no children'
scam_df.loc[scam_df['children'] == 'I want children','children'] = 'want children'
scam_df.loc[scam_df['children'] == 'I have 3 or or more children living elsewhere','children'] = 'more than 2 living elsewhere'
scam_df.loc[scam_df['children'] == 'I have 3 or more children living with me','children'] = 'more than 2 living with me'
scam_df.loc[scam_df['children'] == 'I don’t want children','children'] = "don't want children"
scam_df.loc[scam_df['children'] == 'don’t want children','children'] = "don't want children" #nan

In [ ]:
def update_looking(value):
    if pd.isna(value):
        return value
    if 'Both' in value or 'Male, Female' in value:
        return 'Both'
    return value
scam_df['target'] = scam_df['target'].apply(update_looking)
datingnmore_df['target'] = datingnmore_df['target'].apply(update_looking)

In [ ]:
def check_purposes(df):
    purposes = ['Friendship', 'Romance', 'Serious Relationship', 'Marriage', 'Fun']
    for purpose in purposes:
        # Create a new column for each purpose, assigning 1 if it's present, otherwise 0
        df[f'intent_{purpose.replace(" ", "_")}'] = df['intent'].apply(
            lambda x: 1 if isinstance(x, str) and purpose in x else 0
        )

    return df

scam_df = check_purposes(scam_df)
datingnmore_df = check_purposes(datingnmore_df)

scam_df = scam_df.drop('intent', axis=1)
datingnmore_df = datingnmore_df.drop('intent', axis=1)

In [ ]:
scam_df = scam_df.dropna(subset=['age'])

In [ ]:
scam_df_model = scam_df[['age','target','location','status','children','orientation',
                         'ethnicity','religion','occupation','description','intent_Friendship',
                         'intent_Romance','intent_Serious_Relationship','intent_Marriage','intent_Fun']]

datingnmore_df_model = datingnmore_df[['age','target','location','status','children','orientation',
                         'ethnicity','religion','occupation','description','intent_Friendship',
                         'intent_Romance','intent_Serious_Relationship','intent_Marriage','intent_Fun']]

In [ ]:
scam_df_model['scammer']=1
datingnmore_df_model['scammer']=0

In [ ]:

merge_df = pd.concat([scam_df_model, datingnmore_df_model], ignore_index=True)

In [ ]:
merge_df.shape

(32240, 16)

In [ ]:
def fill_missing_with_mode(df, columns):
    for col in columns:
        mode_value = df[col].mode()[0]
        df[col].fillna(mode_value, inplace=True)
    return df

columns_to_fill = ['status', 'target', 'children','orientation','ethnicity','religion']
merge_df = fill_missing_with_mode(merge_df, columns_to_fill)

In [ ]:
merge_df['city'] = merge_df['location'].str.extract(r'([^,-,-–]+)')[0]
merge_df['city'] = merge_df['city'].str.replace(r'\d+', '', regex=True)
merge_df['city'] = merge_df['city'].str.strip()

# Description Analysis Using MapReduce

In [ ]:
scammer_df_copy = scammer_df.copy()

In [ ]:
desc = scammer_df_copy['description']
desc.head(10)

,description
0,unknown
1,look into the future with hope and would like ...
2,Medical Surgeon. Humble and kind hearted
3,unknown
4,I am a Simple man living my own dreams
5,unknown
6,A matured and responsible woman who is interes...
7,"i,m caring,lovely,honest and good man"
8,I’m interested in meeting a woman to start a f...
9,Am a man of simplicity


In [ ]:
desc.to_csv('desc_input.txt', index=False, header=False)

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import spacy

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')

# NLTK stopwords
nltk_stopwords = set(stopwords.words('english'))

# Scikit-learn stopwords from CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
sklearn_stopwords = set(vectorizer.get_stop_words())

# SpaCy stopwords
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)

# Combine all stopwords into one set (automatically removes duplicates)
combined_stopwords = nltk_stopwords | sklearn_stopwords | spacy_stopwords

# Print the total number of combined stopwords
print(f"Total stopwords combined: {len(combined_stopwords)}")
print(combined_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total stopwords combined: 409
{'among', 'ourselves', 'was', 'mostly', "weren't", 'around', 'will', 'thereafter', 'due', 'by', 'could', 'themselves', 'be', 'o', 'least', 'ltd', 'perhaps', 'how', 'besides', '’d', 'seemed', 'have', 'therefore', "shouldn't", 'name', 'further', 'eg', 'ma', "you'd", 'become', 'un', 'nevertheless', 'thick', 'where', "don't", 'hadn', 'well', 'does', 'of', '’m', 've', 'doing', 'fill', 'ie', "'s", "that'll", 'mine', 'your', 'having', 'system', 'please', 'thereupon', 'keep', 'beforehand', 'same', 'seems', 'therein', 'amount', 'none', 'some', 'about', 'did', "should've", 'together', "you've", 'enough', 'or', 'forty', 'being', 'much', 'him', 'yet', 'whenever', 'each', 'would', 'can', 'hence', 'eleven', 'somehow', '’s', 'serious', 'so', 't', 'only', 'out', 'fire', 'seem', "couldn't", "'ve", 'who', 'an', 'such', 'anyone', '’ve', 'hereafter', 'five', 'you', 'etc', 'has', 'ever', 'we', 'its', '‘s', 'll', 'doesn', 'con', 'thereby', 'the', 'became', 'toward', 'hasn', "mi

In [ ]:
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
!tar -xzf hadoop-3.3.5.tar.gz
# install hadoop to /usr/local
!cp -r hadoop-3.3.5/ /usr/local/
import os
# hadoop needs to know where java is located
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# the following line reduces the amount of Hadoop's printed output, to make it easier to see your own debug output
os.environ["HADOOP_ROOT_LOGGER"] = "WARN,console"

--2024-10-20 04:39:42--  https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706533213 (674M) [application/x-gzip]
Saving to: ‘hadoop-3.3.5.tar.gz.1’

hadoop-3.3.5.tar.gz 100%[===================>] 673.80M  9.23MB/s    in 22s     

2024-10-20 04:40:03 (30.9 MB/s) - ‘hadoop-3.3.5.tar.gz.1’ saved [706533213/706533213]



In [ ]:
#mapper
%%file mapper.py
import sys
import re


combined_stopwords = {'among', 'ourselves', 'was', 'mostly', "weren't", 'around', 'will', 'thereafter', 'due', 'by', 'could', 'themselves', 'be', 'o', 'least', 'ltd', 'perhaps', 'how', 'besides', '’d', 'seemed', 'have', 'therefore', "shouldn't", 'name', 'further', 'eg', 'ma', "you'd", 'become', 'un', 'nevertheless', 'thick', 'where', "don't",'dont','hadn', 'well', 'does', 'of', '’m', 've', 'doing', 'fill', 'ie', "'s", "that'll", 'mine', 'your', 'having', 'system', 'please', 'thereupon', 'keep', 'beforehand', 'same', 'seems', 'therein', 'amount', 'none', 'some', 'about', 'did', "should've", 'together', "you've", 'enough', 'or', 'forty', 'being', 'much', 'him', 'yet', 'whenever', 'each', 'would', 'can', 'hence', 'eleven', 'somehow', '’s', 'serious', 'so', 't', 'only', 'out', 'fire', 'seem', "couldn't", "'ve", 'who', 'an', 'such', 'anyone', '’ve', 'hereafter', 'five', 'you', 'etc', 'has', 'ever', 'we', 'its', '‘s', 'll', 'doesn', 'con', 'thereby', 'the', 'became', 'toward', 'hasn', "mightn't", 'rather', 'under', 'they', 'shan', 'while', 'beyond', 'using', 'do', 'latter', 'however', 'my', 'becomes', '‘d', 'anyhow', 'three', 'again', "hadn't", "you're", 'twelve', 'which', 'thru', 'hers', 'whose', "it's", '’ll', 'just', 'nine', 'full', 'anyway', 'yours', 'n’t', 'noone', 'side', 'used', "won't", 'he', 'm', 'hasnt', 'when', 'never', 'seeming', 'everywhere', 'de', 'if', 'go', 'via', '‘ll', 'than', 'back', 'other', 'interest', 'upon', 'former', 'cry', 'haven', 'what', 'get', 'latterly', 'mill', 'thin', 'find', 'alone', 'couldnt', "mustn't", 'with', "needn't", 'becoming', 'a', "shan't", 'off', "haven't", 'i', 'see', 'am', 'no', 'whereby', 'somewhere', 'at', "aren't", 'next', "'m", "wouldn't", 'either', 'but', 'didn', 'another', 'nowhere', 'others', 'is', 'because', 'very', 'formerly', 'and', 'before', 'hereupon', 'it', 'per', 'up', 'third', 'behind', 'their', 'aren', 'mustn', 'these', 'should', 'wasn', 'meanwhile', "'ll", 'four', '‘ve', 'had', 'nobody', 'once', 'now', 'front', "n't", 'nor', 'elsewhere', 'already', 'through', "doesn't", 'describe', 'sometimes', 'really', 'wherever', 'top', 'still', 'though', 'may', 'anywhere', 'ain', 'made', 'something', 'why', 'few', '’re', 'several', 'n‘t', 'found', 'beside', 'even', 'us', 'herself', "isn't", 'otherwise', 'wouldn', 'take', 'whole', 'although', 'throughout', 'over', 'last', 'theirs', 'on', '‘m', 'shouldn', 'always', 'cant', 'there', "hasn't", 'ca', 'unless', 'regarding', 'since', 'too', 'almost', 'wherein', 'mightn', 'fifteen', 'below', 'own', 'our', 'to', 'weren', 'first', "'d", 'must', 's', 'above', 'part', 'her', 'bill', 'sincere', 'd', 'more', 'whereas', 'herein', 'within', 'whereafter', 'nothing', 'twenty', 'across', 'anything', 'his', 'except', 'empty', 'are', 'as', 'else', 'call', 'yourself', 'onto', 'into', 'whom', 'might', 'most', 'amoungst', 'whoever', 'many', 'various', "she's", 'any', 'someone', 'himself', 'fifty', 'both', 'been', 'often', 'from', "wasn't", 'make', 'yourselves', 'hundred', 'done', 'co', 'whatever', 'me', 'ten', 'put', 'after', 'one', 'cannot', 'all', 'indeed', 'eight', 'myself', 'until', "'re", 'everyone', 'moreover', 'couldn', 'detail', 'afterwards', 'show', 'not', 'down', 'namely', 'move', 'needn', "didn't", 'here', 'then', 'amongst', 'everything', 'less', '‘re', 'quite', 'them', 'hereby', "you'll", 'along', 'between', 'every', 'that', 'towards', 'without', 'sometime', 'for', 'bottom', 'whence', 'were', 'she', 'sixty', 'against', 'this', 'in', 'won', 'thus', 'neither', 'six', 're', 'itself', 'also', 'thence', 'during', 'ours', 'give', 'say', 'inc', 'don', 'y', 'whether', 'whereupon', 'those', 'whither', 'isn', 'two','unknown'}


for line in sys.stdin:
    line = re.sub(r'[^\w\s]', '', line.lower())

    # Split line into words
    words = line.split()

    # Emit word and count (only if not a stopword)
    for word in words:
        if word not in combined_stopwords and word.isalpha():  # check if word is alphabetic
            print(f"{word}\t1")

Overwriting mapper.py


In [ ]:
%%file reducer.py
import sys

counts = {}

# Read input line by line from standard input
for line in sys.stdin:
    word, count = line.strip().split('\t')
    count = int(count)

    # Update word count
    if word in counts:
        counts[word] += count
    else:
        counts[word] = count

# Sort the words by count in descending order
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

# Output the top 30 most frequent words
for word, count in sorted_counts[:30]:
    print(f"{word}\t{count}")

Overwriting reducer.py


In [ ]:
!chmod u+rwx /content/mapper.py
!chmod u+rwx /content/reducer.py

In [ ]:
!rm -rf /content/wordcount_output
!/usr/local/hadoop-3.3.5/bin/hadoop jar /usr/local/hadoop-3.3.5/share/hadoop/tools/lib/hadoop-streaming-3.3.5.jar \
-input desc_input.txt \
-output wordcount_output \
-file /content/mapper.py \
-file /content/reducer.py \
-mapper 'python mapper.py' \
-reducer 'python reducer.py'

2024-10-20 04:57:58,636 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob11779816169570692918.jar tmpDir=null
2024-10-20 04:58:00,037 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-10-20 04:58:03,918 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!


In [ ]:
!cat wordcount_output/part-00000

love	896
im	838
man	652
like	633
life	523
looking	477
good	461
person	384
relationship	368
honest	367
woman	352
caring	314
want	291
know	271
new	260
people	254
loving	243
time	201
heart	195
meet	194
kind	191
going	190
easy	175
single	174
enjoy	166
happy	160
fun	149
things	149
romantic	147
friends	144
